# 16 TensorBoard

In [2]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [64]:
writer = SummaryWriter('runs/mnist2')

In [24]:
from tensorboard import notebook
notebook.display(height=500)

Selecting TensorBoard with logdir runs (started 0:17:08 ago; port 6006, pid 2577).


In [65]:
input_size = 784
hidden_size = 100
num_classes = 10 
nepochs = 5
batch_size = 100
lr = 0.01

In [54]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True,
                                           transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False,
                                           transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                           shuffle=False)

In [55]:
examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [56]:
# for i in range(6):
#     plt.subplot(2, 3, i+1)
#     plt.axis('off')
#     plt.imshow(samples[i][0], cmap='gray')
#plt.show()
img_grid = torchvision.utils.make_grid(samples)
writer.add_image('mnist_images', img_grid)
notebook.display(height=500)

Selecting TensorBoard with logdir runs (started 0:32:40 ago; port 6006, pid 2577).


In [57]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.l2(x)
        return x 

In [66]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [67]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [42]:
writer.add_graph(model, samples.reshape(-1, 28*28))
notebook.display(height=500)

Selecting TensorBoard with logdir runs (started 0:27:39 ago; port 6006, pid 2577).


In [68]:
n_total_steps = len(train_loader)
running_loss = 0.0
running_correct = 0

for epoch in range(nepochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 784)
        
        pred = model(images)
        loss = criteria(pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(pred.data, 1)
        running_correct += (predicted == labels).sum().item()
        
        if i % 100 == 99:
            print(f'epoch {epoch+1} / {nepochs}, step {i+1:4d}/{n_total_steps}, loss {loss.item():.5f}')
            writer.add_scalar('training_loss' , running_loss/100, epoch * n_total_steps + 1)
            writer.add_scalar('training_accuracy', running_correct / 100, epoch * n_total_steps + 1)
            running_loss = 0.0
            running_correct = 0

epoch 1 / 5, step  100/600, loss 0.12036
epoch 1 / 5, step  200/600, loss 0.31708
epoch 1 / 5, step  300/600, loss 0.15334
epoch 1 / 5, step  400/600, loss 0.15641
epoch 1 / 5, step  500/600, loss 0.12765
epoch 1 / 5, step  600/600, loss 0.31821
epoch 2 / 5, step  100/600, loss 0.08055
epoch 2 / 5, step  200/600, loss 0.17511
epoch 2 / 5, step  300/600, loss 0.08944
epoch 2 / 5, step  400/600, loss 0.03753
epoch 2 / 5, step  500/600, loss 0.16982
epoch 2 / 5, step  600/600, loss 0.16683
epoch 3 / 5, step  100/600, loss 0.00995
epoch 3 / 5, step  200/600, loss 0.08254
epoch 3 / 5, step  300/600, loss 0.05785
epoch 3 / 5, step  400/600, loss 0.07539
epoch 3 / 5, step  500/600, loss 0.03904
epoch 3 / 5, step  600/600, loss 0.03657
epoch 4 / 5, step  100/600, loss 0.03127
epoch 4 / 5, step  200/600, loss 0.02130
epoch 4 / 5, step  300/600, loss 0.02886
epoch 4 / 5, step  400/600, loss 0.16668
epoch 4 / 5, step  500/600, loss 0.11450
epoch 4 / 5, step  600/600, loss 0.06069
epoch 5 / 5, ste

In [44]:
notebook.display()

Selecting TensorBoard with logdir runs (started 0:29:15 ago; port 6006, pid 2577).


In [70]:
labels_ = []
preds = []

with torch.no_grad():
    correct = 0
    samples = 0 
    for images, labels in test_loader:
        images = images.reshape(-1, 784)
        pred = model(images)
        
        _, predictions = torch.max(pred, 1)
        samples += labels.shape[0]
        correct += (predictions == labels).sum().item()
        
        labels_.append(predictions)
        class_prediction = [nn.functional.softmax(output, dim=0) for output in pred]
        preds.append(class_prediction)
        
    labels = torch.cat(labels_)
    preds = torch.cat([torch.stack(batch) for batch in preds])

In [71]:
for k in range(10):
    label_k = labels == k 
    preds_k = preds[:, k]
    writer.add_pr_curve(str(k), label_k, preds_k)


In [72]:
notebook.display(height=500)

Selecting TensorBoard with logdir runs (started 0:42:11 ago; port 6006, pid 2577).
